In [14]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, SimpleRNN, LSTM
from sklearn.metrics import mean_squared_error, r2_score
import math
import time

In [15]:
def load_and_preprocess_data(url, split_percent=0.8):
    df = pd.read_csv(url, usecols=[1], engine='python')
    data = np.array(df.values.astype('float32'))
    scaler = MinMaxScaler(feature_range=(0, 1))
    data = scaler.fit_transform(data).flatten()
    n = len(data)
    split = int(n * split_percent)
    train_data = data[:split]
    test_data = data[split:]
    return train_data, test_data, data

In [16]:
def create_feedforward_nn(input_shape, activation):
    model = Sequential()
    
    model.add(Dense(64, input_shape=input_shape, activation=activation))
    model.add(Flatten())
    
    model.add(Dense(32, activation=activation))
    model.add(Flatten())
    
    model.add(Dense(1, activation=activation))
    model.add(Flatten())
    
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [17]:
def create_rnn_model(hidden_units, dense_units, input_shape, activation):
    model = Sequential()

    model.add(SimpleRNN(hidden_units, input_shape=input_shape, activation=activation))

    model.add(Dense(units=dense_units, activation=activation))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [18]:

def create_lstm_model(hidden_units, dense_units, input_shape, activation):
    model = Sequential()

    model.add(LSTM(hidden_units, input_shape=input_shape, activation=activation))

    model.add(Dense(units=dense_units, activation=activation))

    model.compile(loss='mean_squared_error', optimizer='adam')

    return model

In [19]:
def prepare_sequence_data(dat, time_steps):
    indY = np.arange(time_steps, len(dat), time_steps)
    Y = dat[indY]

    rowsX = len(Y)
    X = dat[range(time_steps * rowsX)]
    X = np.reshape(X, (rowsX, time_steps, 1))

    return X, Y

In [20]:
def evaluate_and_print_metrics(trainY, testY, trainPredict, testPredict):
    train_rmse = math.sqrt(mean_squared_error(trainY, trainPredict))
    test_rmse = math.sqrt(mean_squared_error(testY, testPredict))
    train_r2 = r2_score(trainY, trainPredict)
    test_r2 = r2_score(testY, testPredict)

    print('Train RMSE: %.3f' % train_rmse)
    print('Test RMSE: %.3f' % test_rmse)
    print('Train R2: %.3f' % train_r2)
    print('Test R2: %.3f' % test_r2)


In [21]:
time_steps = 12
sunspot_data_url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-sunspots.csv'
train_data, test_data, all_data = load_and_preprocess_data(sunspot_data_url)

train_X, train_Y = prepare_sequence_data(train_data, time_steps)
test_X, test_Y = prepare_sequence_data(test_data, time_steps)

nn_model = create_feedforward_nn(input_shape=(time_steps, 1), activation='relu')
rnn_model = create_rnn_model(hidden_units=3, dense_units=1, input_shape=(time_steps, 1), activation='tanh')
lstm_model = create_lstm_model(hidden_units=3, dense_units=1, input_shape=(time_steps, 1), activation='tanh')

time_start_nn = time.time()
nn_model.fit(train_X, train_Y, epochs=20, batch_size=1, verbose=2)
time_end_nn = time.time()

time_start_rnn = time.time()
rnn_model.fit(train_X, train_Y, epochs=20, batch_size=1, verbose=2)
time_end_rnn = time.time()

time_start_lstm = time.time()
lstm_model.fit(train_X, train_Y, epochs=20, batch_size=1, verbose=2)
time_end_lstm = time.time()

train_predict_nn = nn_model.predict(train_X)
test_predict_nn = nn_model.predict(test_X)

train_predict_rnn = rnn_model.predict(train_X)
test_predict_rnn = rnn_model.predict(test_X)

train_predict_lstm = lstm_model.predict(train_X)
test_predict_lstm = lstm_model.predict(test_X)

print('\nFeedforward NN')
evaluate_and_print_metrics(train_Y, test_Y, train_predict_nn, test_predict_nn)
print('Training Time: %.2f seconds' % (time_end_nn - time_start_nn))

print('\nRNN Model')
evaluate_and_print_metrics(train_Y, test_Y, train_predict_rnn, test_predict_rnn)
print('Training Time: %.2f seconds' % (time_end_rnn - time_start_rnn))

print('\nLSTM Model')
evaluate_and_print_metrics(train_Y, test_Y, train_predict_lstm, test_predict_lstm)
print('Training Time: %.2f seconds' % (time_end_lstm - time_start_lstm))

Epoch 1/20
187/187 - 1s - loss: 0.0492 - 1s/epoch - 7ms/step
Epoch 2/20
187/187 - 0s - loss: 0.0492 - 327ms/epoch - 2ms/step
Epoch 3/20
187/187 - 0s - loss: 0.0492 - 257ms/epoch - 1ms/step
Epoch 4/20
187/187 - 0s - loss: 0.0492 - 266ms/epoch - 1ms/step
Epoch 5/20
187/187 - 0s - loss: 0.0492 - 245ms/epoch - 1ms/step
Epoch 6/20
187/187 - 0s - loss: 0.0492 - 247ms/epoch - 1ms/step
Epoch 7/20
187/187 - 0s - loss: 0.0492 - 226ms/epoch - 1ms/step
Epoch 8/20
187/187 - 0s - loss: 0.0492 - 219ms/epoch - 1ms/step
Epoch 9/20
187/187 - 0s - loss: 0.0492 - 217ms/epoch - 1ms/step
Epoch 10/20
187/187 - 0s - loss: 0.0492 - 218ms/epoch - 1ms/step
Epoch 11/20
187/187 - 0s - loss: 0.0492 - 221ms/epoch - 1ms/step
Epoch 12/20
187/187 - 0s - loss: 0.0492 - 224ms/epoch - 1ms/step
Epoch 13/20
187/187 - 0s - loss: 0.0492 - 223ms/epoch - 1ms/step
Epoch 14/20
187/187 - 0s - loss: 0.0492 - 272ms/epoch - 1ms/step
Epoch 15/20
187/187 - 0s - loss: 0.0492 - 362ms/epoch - 2ms/step
Epoch 16/20
187/187 - 0s - loss: 0.04

## (NN):

### Ventajas:

Es mas simple de utilzar, ademas que esta puede funcionar de mejor manera con problemas mas sencillos, con una relacion entre entradas y salidas simples o con patrones no tan complejos. Ademas que el entrenamiento de este es mucho mas simple y veloz en comparacion con las otras dos redes

### Desventajas

Este no tiene una capacidad de retencion propieamente dicho, osea que no puede capturar los patrones de forma real. Ademas que este no es muy adecuado para secuencias, porque esta no maneja datos de series temporales de forma buena por lo antes mencionado.

## (RNN):

### Pros:

Las redes neuronales recurrentes están diseñadas específicamente para tratar con secuencias y tienen la capacidad de capturar dependencias temporales y patrones en datos secuenciales. Son capaces de manejar secuencias de longitud variable y pueden adaptarse a diferentes longitudes de entrada.

### Cons:

No obstante, las RNNs pueden sufrir del problema del gradiente desvaneciente, lo que puede dificultar su entrenamiento y afectar su capacidad para capturar patrones a largo plazo. También pueden tener dificultades para mantener dependencias a largo plazo debido a este problema. Además, el entrenamiento de RNNs puede ser más lento en comparación con las redes neuronales feedforward debido a la naturaleza secuencial de su procesamiento.

## (LSTM):

### Pros:

Las redes LSTM se destacan en capturar dependencias a largo plazo en secuencias, abordando el problema del gradiente desvaneciente presente en RNNs tradicionales. Tienen la capacidad de retener información y patrones en secuencias más largas, lo que las hace ideales para datos secuenciales complejos.

### Cons:

Sin embargo, las LSTMs son más complejas que las RNNs y las redes neuronales feedforward básicas, lo que puede resultar en tiempos de entrenamiento más prolongados. Si no se regularizan adecuadamente, las LSTMs pueden sobreajustar en conjuntos de datos pequeños.

Basándonos en los resultados de evaluación y consideraciones teóricas, la arquitectura LSTM parece ser la elección más adecuada para resolver el problema de pronóstico de manchas solares. Aunque las LSTMs pueden tener un tiempo de entrenamiento más largo, su capacidad para capturar patrones complejos y dependencias temporales en los datos es fundamental para un pronóstico preciso. Dado que el conjunto de datos de manchas solares implica relaciones secuenciales significativas, la arquitectura LSTM es altamente apropiada para este caso.